In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [2]:
api_key = 'pcsk_4ULNwR_2DBn2Uwwz8HV1CGaqKozndhJPJggKnrqNckfyPBWoTUcLKTUipot6v7MKomsscP'

In [3]:
!pip install langchain_community

In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
pip install pinecone

In [7]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_4ULNwR_2DBn2Uwwz8HV1CGaqKozndhJPJggKnrqNckfyPBWoTUcLKTUipot6v7MKomsscP")

In [8]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [9]:
index=pc.Index(index_name)
index

In [14]:
!pip install langchain_huggingface

In [15]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
sentences=[
    "In 2023, I watched Attack on Titan: The Final Season"

    "In 2022, I watched Demon Slayer: Entertainment District Arc"

    "In 2021, I watched Jujutsu Kaisen",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [19]:
retriever


PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x798a678575c0>, index=<pinecone.db_data.index.Index object at 0x798a67733410>)

In [22]:
retriever.add_texts(
    [
    "In 2023, I watched Attack on Titan: The Final Season",

    "In 2022, I watched Demon Slayer: Entertainment District Arc",

    "In 2021, I watched Jujutsu Kaisen",

]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
retriever.invoke("Which anime did i watch 2022")

[Document(metadata={'score': 0.371315241}, page_content='In 2021, I watched Jujutsu Kaisen'),
 Document(metadata={'score': 0.495565355}, page_content='In 2022, I watched Demon Slayer: Entertainment District Arc'),
 Document(metadata={'score': 0.38602221}, page_content='In 2023, I watched Attack on Titan: The Final Season')]